In [1]:
import fnmatch
import pandas as pd
from scipy.optimize import linprog

In [3]:
gatewayVolumesLoc = "./GatewayVolumes_2056.csv"
postcodeVolumesLoc = "./INITFMCC_PCVol_Y2056_WFY_Funded_Op2_001.CSV"
travelTimesCompleteLoc = "./INITFMCC_TTByGateway_Y2056_WFY_Funded_Op2_001.CSV"

In [13]:
gatewayVolumes = pd.read_csv(gatewayVolumesLoc)
postcodeVolumes = pd.read_csv(postcodeVolumesLoc)
allTravelTimes = pd.read_csv(travelTimesCompleteLoc)
gatewayMatchColNames = ['Gateway','Sector','Import / Export','Full / Empty']

spreadWeight = 2000

In [18]:
postcodeVolumes

,Gateway,Sector,Import / Export,Full / Empty,Industry Class,To,To Postcode,To State,Input Volume,Output Volume
0,Swanson Dock West,I&MC,Import,Full,IMEX,Container Parks,3003,Complete,239.223621,NaN
1,Swanson Dock West,I&MC,Import,Full,IMEX,Container Parks,3011,Complete,304.105831,NaN
2,Swanson Dock West,I&MC,Import,Full,IMEX,Container Parks,3012,Complete,1031.777174,NaN
3,Swanson Dock West,I&MC,Import,Full,IMEX,Container Parks,3013,Complete,32.208107,NaN
4,Swanson Dock West,I&MC,Import,Full,IMEX,Container Parks,3018,Complete,187.390876,NaN
...,...,...,...,...,...,...,...,...,...,...
1674,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,Transport Depots,3061,Full,8.384297,8.384297
1675,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,Transport Depots,3062,Empty,NaN,37.130458
1676,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,Transport Depots,3062,Full,5.030578,5.030578
1677,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,Transport Depots,3753,Empty,NaN,148.521832


In [5]:
# this function separates strings by semicolon and matches with wildcard
def matchStrings(primaryString, secondaryString):
    isMatch = False
    secondaryStringList = secondaryString.split(';')
    for i in range(len(secondaryStringList)):
        isMatch = isMatch | fnmatch.fnmatch(primaryString, secondaryStringList[i])
    return isMatch
    


# this function carries out matchStrings for all required columns to get a full match
def matchCols(primaryRow, secondaryRow, primaryColsList, secondaryColsList):
    rowMatch = True
    for j in range(len(primaryColsList)):
        rowMatch = rowMatch & matchStrings(primaryRow[primaryColsList[j]], secondaryRow[secondaryColsList[j]])
    return rowMatch

In [6]:
allSolutions = pd.DataFrame()

In [15]:
from IPython.display import display
from scipy.optimize import linprog

def pretty_print_df(df, title="DataFrame", max_rows=10):
    """Display a DataFrame with a title and limited rows"""
    print(f"\n{'='*50}\n{title}\n{'='*50}")
    display(df.head(max_rows))
    print(f"Shape: {df.shape}")
    if len(df) > max_rows:
        print(f"Showing first {max_rows} rows of {len(df)} total rows")

allSolutions = pd.DataFrame()

for gatewayIt in range(len(gatewayVolumes)):
    # get current row
    gatewayRow = gatewayVolumes.iloc[gatewayIt]
    print('\nCurrent gateway:' + str(gatewayRow.values) + '\n')

    # get postcode volumes and travel times that match the current gateway
    matchedFlows = postcodeVolumes[postcodeVolumes.apply(lambda row: matchCols(gatewayRow, row, gatewayMatchColNames, gatewayMatchColNames), axis=1)]
    pretty_print_df(matchedFlows, "Matched Flows")

    matchedTimes = allTravelTimes[allTravelTimes.apply(lambda row: matchCols(gatewayRow, row, gatewayMatchColNames, gatewayMatchColNames), axis=1)].reset_index()
    pretty_print_df(matchedTimes, "Matched Times")

    # check there are chain flow volumes for the gateway
    if len(matchedTimes) == 0:
        print('INPUT DATA ERROR: No chain flow volumes corresponding to this gateway\n')
        continue

    # get input and output marginals
    inputMarginals = matchedFlows.sort_values(by=['To Postcode','To','To State']).reset_index()\
        .filter(['To Postcode','To','To State','Input Volume'])\
        .rename(columns={'Input Volume':'Volume'})
    inputMarginals = inputMarginals.drop(inputMarginals[inputMarginals['Volume'] == 0].index)\
        .dropna().reset_index()
    pretty_print_df(inputMarginals, "Input Marginals")

    outputMarginals = matchedFlows.sort_values(by=['To Postcode','To','To State']).reset_index()\
        .filter(['To Postcode','To','To State','Output Volume'])\
        .rename(columns={'Output Volume':'Volume'})
    outputMarginals = outputMarginals.drop(outputMarginals[outputMarginals['Volume'] == 0].index)\
        .dropna().reset_index()
    pretty_print_df(outputMarginals, "Output Marginals")

    # get index references for tables
    inputMarginals['Input Index'] = inputMarginals.index
    outputMarginals['Output Index'] = outputMarginals.index
    matchedTimes['Time Index'] = matchedTimes.index

    print('Merge marginals and travel times...\n')
    timesMarginalsMatch = matchedTimes.merge(inputMarginals, 
                                           left_on=['To Postcode','To','To State'], 
                                           right_on=['To Postcode','To','To State'], 
                                           how='left')\
                                     .merge(outputMarginals, 
                                           left_on=['From Postcode','From','From State'], 
                                           right_on=['To Postcode','To','To State'], 
                                           how='left')\
                                     .filter(gatewayMatchColNames + ['Run Type','From Postcode','From',
                                                                   'To Postcode_x','To_x','From State',
                                                                   'Travel Time','Time Index','Input Index',
                                                                   'Output Index'])\
                                     .rename(columns={'To Postcode_x':'To Postcode', 
                                                    'To_x':'To',
                                                    'From State':'State'})
    timesMarginalsMatch['Travel Time'] = timesMarginalsMatch['Travel Time'].fillna(value=240)
    pretty_print_df(timesMarginalsMatch, "Times Marginals Match")

    # generate inequality constraints for auxiliary z-variables
    nDecisionVariables = len(timesMarginalsMatch) + len(outputMarginals)
    lhs_geq = []
    for linkInd in timesMarginalsMatch.index:
        newConstraint = [0] * nDecisionVariables
        newConstraint[len(timesMarginalsMatch) + timesMarginalsMatch.loc[linkInd, 'Output Index']] = -1
        newConstraint[linkInd] = 1 / outputMarginals.loc[timesMarginalsMatch.loc[linkInd, 'Output Index'], 'Volume']
        lhs_geq.append(newConstraint)
    
    rhs_geq = [0] * len(lhs_geq)

    # generate constraints
    lhs_eq = []
    for outputInd in range(len(outputMarginals)):
        ind = set(timesMarginalsMatch[timesMarginalsMatch['Output Index'] == outputInd]['Time Index'])
        lhs_eq.append([int(i in ind) for i in range(nDecisionVariables)])

    for inputInd in range(len(inputMarginals)):
        ind = set(timesMarginalsMatch[timesMarginalsMatch['Input Index'] == inputInd]['Time Index'])
        lhs_eq.append([int(i in ind) for i in range(nDecisionVariables)])

    obj = timesMarginalsMatch['Travel Time'].tolist()
    obj.extend([spreadWeight] * len(outputMarginals))
    rhs_eq = list(pd.concat([outputMarginals, inputMarginals])['Volume'])
    bnd = [(0, float('inf'))] * len(obj)

    # check for an error arising from incorrect optimisation problem dimensions 
    if len(lhs_eq) != len(rhs_eq):
        print('ERROR: different number of constraints on LHS to RHS.\n')
        allSolutions = pd.DataFrame({'error':['']})
        break
    if len(obj) != len(lhs_eq[0]):
        print('ERROR: constraints different length to objective function.\n')
        allSolutions = pd.DataFrame({'error':['']})
        break


    opt = linprog(c=obj, A_eq=lhs_eq, b_eq=rhs_eq, A_ub=lhs_geq, b_ub=rhs_geq,\
                  method='simplex', options={'rr':False, 'tol':0.000001, 'maxiter':10000})
    
    # check for an error arising from negative solutions
    if min(opt.x) < 0:
        print('EXECUTION ERROR: OPTIMISATION FAILED - please update scipy package to version 0.18.0 or greater.\n')
        allSolutions = pd.DataFrame({'error':['']})
        break
    else:
        print('Result: ' + opt.message + '\n')

    # check for an error arising from feasibility
    if opt.status == 2:
        print('INPUT DATA ERROR: Optimisation problem is infeasible\n')
        allSolutions = pd.DataFrame({'error':['']})
        break

    # check for an error arising from incorrect auxiliary variables
    auxCheck = opt.x[len(matchedTimes):]
    if max(auxCheck) > 1.001:
        print('EXECUTION ERROR: OPTIMISATION FAILED - please update scipy package to version 0.18.0 or greater.\n')
        allSolutions = pd.DataFrame({'error':['']})
        break

    # arrange output table
    matchedTimes['Volume'] = opt.x[:len(matchedTimes)]
    matchedTimes = matchedTimes.filter(gatewayMatchColNames + ['Industry Class','Run Type',
                                                             'From Postcode','To Postcode',
                                                             'From','From State','To','Volume'])\
                              .rename(columns={'Volume':'TEUs'})
    pretty_print_df(matchedTimes, "Optimized Matched Times")

    # combine with solutions from other gateways using concat instead of append
    allSolutions = pd.concat([allSolutions, matchedTimes])
    pretty_print_df(allSolutions, "Cumulative Solutions")


Current gateway:['Swanson Dock West' 'I&MC' 'Import' 'Full' 'IMEX' 4209.470819953]


Matched Flows


,Gateway,Sector,Import / Export,Full / Empty,Industry Class,To,To Postcode,To State,Input Volume,Output Volume
0,Swanson Dock West,I&MC,Import,Full,IMEX,Container Parks,3003,Complete,239.223621,NaN
1,Swanson Dock West,I&MC,Import,Full,IMEX,Container Parks,3011,Complete,304.105831,NaN
2,Swanson Dock West,I&MC,Import,Full,IMEX,Container Parks,3012,Complete,1031.777174,NaN
3,Swanson Dock West,I&MC,Import,Full,IMEX,Container Parks,3013,Complete,32.208107,NaN
4,Swanson Dock West,I&MC,Import,Full,IMEX,Container Parks,3018,Complete,187.390876,NaN
5,Swanson Dock West,I&MC,Import,Full,IMEX,Container Parks,3025,Complete,411.685040,NaN
6,Swanson Dock West,I&MC,Import,Full,IMEX,Container Parks,3207,Complete,529.125410,NaN
7,Swanson Dock West,I&MC,Import,Full,IMEX,Importers,3003,Complete,9.986704,NaN
8,Swanson Dock West,I&MC,Import,Full,IMEX,Importers,3003,Empty,212.932256,212.932256
9,Swanson Dock West,I&MC,Import,Full,IMEX,Importers,3011,Complete,13.722559,NaN


Shape: (127, 10)
Showing first 10 rows of 127 total rows

Matched Times


,index,Gateway,Sector,Import / Export,Full / Empty,Industry Class,Run Type,From Postcode,From,From State,To Postcode,To,To State,Travel Time
0,0,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3175,Transport Depots,Empty,3003,Swanson Dock West,Complete,34.712602
1,1,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3003,Transport Depots,Empty,3003,Swanson Dock West,Complete,1.856365
2,2,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3043,Transport Depots,Empty,3003,Swanson Dock West,Complete,24.648563
3,3,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3012,Transport Depots,Empty,3003,Swanson Dock West,Complete,16.510005
4,4,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3025,Transport Depots,Empty,3003,Swanson Dock West,Complete,21.248426
5,5,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3026,Transport Depots,Empty,3003,Swanson Dock West,Complete,21.867616
6,6,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3029,Transport Depots,Empty,3003,Swanson Dock West,Complete,30.164240
7,7,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3029,PRS Terminals WIFT / BIFT,Empty,3003,Swanson Dock West,Complete,30.164240
8,8,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3029,PRS Terminals WIFT / BIFT,Empty,3003,Swanson Dock West,Complete,30.164240
9,9,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3753,PRS Terminals WIFT / BIFT,Empty,3003,Swanson Dock West,Complete,54.080254


Shape: (2098, 14)
Showing first 10 rows of 2098 total rows

Input Marginals


...

/var/folders/fg/wfh8sp4x7lbb49hgnww7yrlm0000gn/T/ipykernel_9928/2523735054.py:107: OptimizeWarning: The pivot operation produces a pivot value of: 8.1e-03, which is only slightly greater than the specified tolerance 1.0e-06. This may lead to issues regarding the numerical stability of the simplex method. Removing redundant constraints, changing the pivot strategy via Bland's rule or increasing the tolerance may help reduce the issue.
  opt = linprog(c=obj, A_eq=lhs_eq, b_eq=rhs_eq, A_ub=lhs_geq, b_ub=rhs_geq,\
/var/folders/fg/wfh8sp4x7lbb49hgnww7yrlm0000gn/T/ipykernel_9928/2523735054.py:107: OptimizeWarning: The pivot operation produces a pivot value of: 7.0e-03, which is only slightly greater than the specified tolerance 1.0e-06. This may lead to issues regarding the numerical stability of the simplex method. Removing redundant constraints, changing the pivot strategy via Bland's rule or increasing the tolerance may help reduce the issue.
  opt = linprog(c=obj, A_eq=lhs_eq, b_eq=rhs_e

,Gateway,Sector,Import / Export,Full / Empty,Industry Class,Run Type,From Postcode,To Postcode,From,From State,To,TEUs
0,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,IG,3753,3753,Transport Depots,Empty,BIFT South Rail Terminal,20.338125
1,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,IG,3061,3753,Transport Depots,Empty,BIFT South Rail Terminal,0.000000
2,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,IG,3062,3753,Transport Depots,Empty,BIFT South Rail Terminal,0.000000
3,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,IG,3753,3753,Transport Depots,Empty,BIFT South Rail Terminal,49.507277
4,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,IG,3061,3753,Transport Depots,Empty,BIFT South Rail Terminal,0.000000
5,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,IG,3062,3753,Transport Depots,Empty,BIFT South Rail Terminal,0.000000
6,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,CR,3003,3753,Exporters,Full,BIFT South Rail Terminal,0.139948
7,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,CR,3012,3753,Exporters,Full,BIFT South Rail Terminal,10.387714
8,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,CR,3013,3753,Exporters,Full,BIFT South Rail Terminal,8.568837
9,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,CR,3018,3753,Exporters,Full,BIFT South Rail Terminal,2.770540


Shape: (281, 12)
Showing first 10 rows of 281 total rows

Cumulative Solutions


,Gateway,Sector,Import / Export,Full / Empty,Industry Class,Run Type,From Postcode,To Postcode,From,From State,To,TEUs
0,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3175,3003,Transport Depots,Empty,Swanson Dock West,70.680413
1,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3003,3003,Transport Depots,Empty,Swanson Dock West,749.008581
2,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3043,3003,Transport Depots,Empty,Swanson Dock West,24.172279
3,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3012,3003,Transport Depots,Empty,Swanson Dock West,165.932801
4,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3025,3003,Transport Depots,Empty,Swanson Dock West,20.978087
5,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3026,3003,Transport Depots,Empty,Swanson Dock West,0.000000
6,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3029,3003,Transport Depots,Empty,Swanson Dock West,7.178356
7,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3029,3003,PRS Terminals WIFT / BIFT,Empty,Swanson Dock West,59.659188
8,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3029,3003,PRS Terminals WIFT / BIFT,Empty,Swanson Dock West,59.659188
9,Swanson Dock West,I&MC,Import,Full,IMEX,RL,3753,3003,PRS Terminals WIFT / BIFT,Empty,Swanson Dock West,13.586465


Shape: (20548, 12)
Showing first 10 rows of 20548 total rows


In [16]:
allSolutions

,Gateway,Sector,Import / Export,Full / Empty,Industry Class,Run Type,From Postcode,To Postcode,From,From State,To,TEUs
0,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3175,3003,Transport Depots,Empty,Swanson Dock West,70.680413
1,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3003,3003,Transport Depots,Empty,Swanson Dock West,749.008581
2,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3043,3003,Transport Depots,Empty,Swanson Dock West,24.172279
3,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3012,3003,Transport Depots,Empty,Swanson Dock West,165.932801
4,Swanson Dock West,I&MC,Import,Full,IMEX,BR,3025,3003,Transport Depots,Empty,Swanson Dock West,20.978087
...,...,...,...,...,...,...,...,...,...,...,...,...
276,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,CR,3753,3201,BIFT South Rail Terminal,Empty,Exporters,0.858138
277,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,CR,3753,3207,BIFT South Rail Terminal,Empty,Exporters,0.000000
278,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,CR,3753,3214,BIFT South Rail Terminal,Empty,Exporters,0.535706
279,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,CR,3753,3458,BIFT South Rail Terminal,Empty,Exporters,0.858138
